In [98]:
import sympy as sp
import scipy as sc


def solve(classical, quantum, hardware_slowdown):

    classical = sp.simplify(classical)
    quantum = sp.simplify(quantum)
    hardware_slowdown = sp.simplify(hardware_slowdown)

    n = sp.symbols('n')
    solutions = sp.solve((classical) - ((quantum) * (hardware_slowdown)), n)
    solutions = [sp.N(solution) for solution in solutions]
    solutions = [float(solution) for solution in solutions if solution.is_real]
    solutions.sort()
    return solutions


def is_valid(expr):
    try:
        n = sp.symbols('n')
        sympy_expr = sp.sympify(expr)
        for sub_expr in sympy_expr.atoms(sp.Function):
            if isinstance(sub_expr, sp.core.function.AppliedUndef):
                return False

        return sympy_expr.free_symbols == {n} or sympy_expr.free_symbols == set()
    except sp.SympifyError:
        return False


def get_points(expr, midPoint, start=0, inverted=False):
    points = []
    n = sp.symbols('n')
    for i in range(start, 200):
        x = midPoint ** (i / 100)
        y = sp.sympify(expr).subs(n, x)
        if inverted:
            points.append([float(y), float(x)])
        else:
            points.append([float(x), float(y)])
    return sorted(points)


def get_quantum_advantage_data(classical, quantum, hardware_slowdown):
    validation = [is_valid(classical), is_valid(
        quantum), is_valid(hardware_slowdown)]
    if False in validation:
        return None
    solutions = solve(classical, quantum, hardware_slowdown)
    nStar = 1
    if len(solutions) > 0:
        nStar = solutions[-1] if solutions[-1] > 1 else float(sp.oo)
    midPoint = nStar if nStar > 1 and nStar < sp.oo else 1000000000000

    classical_points = get_points(classical, midPoint)

    quantum = sp.simplify(quantum)
    hardware_slowdown = sp.simplify(hardware_slowdown)
    quantumXHardware = sp.simplify(quantum * hardware_slowdown)
    quantum_points = get_points(quantumXHardware, midPoint)
    n = sp.symbols('n')
    step_star = sp.simplify(quantumXHardware).subs(n, nStar)
    return {
        "n_star": nStar,
        "step_star": float(step_star),
        "classical_steps": classical_points,
        "quantum_steps": quantum_points
    }


def get_quantum_economic_advantage_data(
        classical_runtime,
        quantum_runtime,
        hardware_slowdown,
        current_year=2023,
        quantum_improvement_rate=2,
        physical_logical_qubits_ratio=1000,
        current_physical_qubits=4158):
    n = sp.Symbol('n')

    quantum_feasible = sp.simplify(
        f'log( (({physical_logical_qubits_ratio})*log( (n) , 2))/({current_physical_qubits}) , 2) + ({current_year})')
    quantum_advantage = sp.simplify(f'log( (({hardware_slowdown})*({quantum_runtime})/({
                                    classical_runtime}) ), ({quantum_improvement_rate})) + ({current_year})')

    lam_f = sp.lambdify(n, sp.simplify(quantum_feasible - quantum_advantage))

    intersection = sp.nsolve(quantum_feasible - quantum_advantage, n, 2)
    t = quantum_feasible.subs(n, intersection).evalf()

    

    return {
        "n_star": (intersection),
        "t_star": float(t),
        # "quantum_feasible": get_points(quantum_feasible, intersection, start=1, inverted=True),
        # "quantum_advantage": get_points(quantum_advantage, intersection, start=1, inverted=True)
    }

get_quantum_economic_advantage_data('n', 'sqrt(n)', '10^6', current_year=2023, quantum_improvement_rate=2, physical_logical_qubits_ratio=1000000, current_physical_qubits=4158)


{'n_star': 67225.9964086344,
 't_star': 2034.91320276277,
 'quantum_feasible': [[2028.2693465729953, 1.1175716402200258],
  [2029.2693465729953, 1.2489663710240788],
  [2029.8543090737164, 1.3958093958450333],
  [2030.2693465729953, 1.559916995949057],
  [2030.5912746678828, 1.7433189957698831],
  [2030.8543090737164, 1.9482838695292766],
  [2031.076701495053, 2.1773467996840528],
  [2031.2693465729953, 2.4333410342507307],
  [2031.4392715744377, 2.719432930862283],
  [2031.5912746678828, 3.039161121012114],
  [2031.7287781916327, 3.3964802789024406],
  [2031.8543090737164, 3.795810036267971],
  [2031.9697862911364, 4.242089648195632],
  [2032.076701495053, 4.740839086094386],
  [2032.1762371686038, 5.29822731346571],
  [2032.2693465729953, 5.921148588968415],
  [2032.3568094142456, 6.617307740559924],
  [2032.4392715744377, 7.395315465458226],
  [2032.5172740864389, 8.264794834676675],
  [2032.5912746678828, 9.23650031947161],
  [2032.661663995774, 10.322450811924677],
  [2032.7287781